# Texture Synthesis Preparation: Classification with SVM

<h4>Application and analysis of texture classification using SVM against dataset of differing textures, in order to prepare for Neural Network based Texture Synthesis, Procedural Texture generation, and applications of Computer Vision object recognition.</h4>

## Problem: Texture Synthesis Requires Pre-Classification

<h2>What is Texture Synthesis?</h2>
<h4><b><i>Texture Synthesis::</i></b>
<br>
Texture Synthesis is the process of generating novel textures from samples of existing textures and imagery, while retaining the visual elements of the original. 
Texture Synthesis can be used to in 2D and 3D image generation applications, such as::</h4>

<h5>Extend existing images and textures mathematically such as procedural textures to solve 2D image tiling</h5>
<img src ="img/TextureTiling.png" alt="TextureTiling.png" width="500" height="400">
<br>
<i>Blender Guru, "Blender Texturing for Beginners - Tutorial"<i>

<h5>Move materials from one type of material to another in existing images in Texture Transfer</h5>
<img src ="img/TextureTransfer.png" alt="TextureTransfer.png" width="500" height="400">
<br>
<i>University of Illinois, "Textureshop: Texture Synthesis as as Photograph Editing Tool"<i>

<h5>Shift portions or entirety of textures from one art style to another via Quilting or Image Analogies and Artistic Filters.</h5>
<img src ="img/TextureAImageAnalogy.png" alt="TextureAnalogy.png" width="500" height="400">
<br>
<i>University of Washington, "Texture Synthesis"<i>

<h3><b>And many more applications!</b></h3>
<h4>Textures generated via synthesis methods through these approaches are known as Synthetic Textures, and are a precursory method to creating procedural textures in large scale 3D applications.</h4> 

<h2><b><i>Summary of Goal:: </b></i>Explore the preparation of training classification models on existing textures, in preparation for later Neural Network Synthetic Texture Genesis.</h2>

## Exploration of the Image Collection

<h4><b><i>Origin of the Textures Dataset::</i></b>
<br>
This data all came from hand, from me walking about my apartment complex taking photos like a total weirdo - we'll call this scientific process <b>Photogrammetric Loitering</b></h4>
<li>Size of the dataset can be represented as varying classes of textures depending on how many models and which kinds are used</li>
<li>Dataset comprises 5 directories with 8 images for each type, and one 'other' folder for later testing - it is a mixture of other texture types</li>
<li>Data cleaning in this project began in this controlled case by my own picking and deleting unwanted or unused textures by hand prior to training</li>

<h4><b><i>Importing and Exploring the Data::</i></b></h4>

In [9]:
# Importing the packages
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from PIL import Image, ImageOps
from skimage.feature import local_binary_pattern
from sklearn import preprocessing, metrics
from sklearn.svm import LinearSVC

<h4><b><i>Pull the Dataset from .Zip Folder 'Textures'::</i></b></h4>

In [ ]:
zip_file = 'img/grasses.zip'
# (?) Should I use my model on grass vs leaves, concrete vs brick, and another one some place?
with zipfile.ZipFile(zip_file, 'r') as images:
    images.extractall('./')

<h4><b><i>Perform data analysis and investigate the data, visualize relationship between classes::</i></b></li>

## Preparing the Data For Processing

talk about
Apply relevant data transformation (i.e., normalization or encoding etc) for the variables
- If the dataset is impbalanced, apply techniques such as oversampling and undersampling, SMOTE (synthetic minority over sampling technique) etc

In [ ]:
# Creating Local Binary Patterns for the pixels.

## Model Building

### Jump into the conversation with a number of papers
Justify selection of algorithm, and train multiple models (min of 2) and compare their performances
Tune some hyperparameters like grid serach, random search, bayesian optimization.

## Evaluation

### Maybe some stuff in Blender or Substance
evaluate model performance (accuracy, precision, recall)
perform cross-validation and report average performance (across k-fold cross validation or some such)

## Conclusions and Next Steps

discuss insights drawn from the model, how does it solve the problem i set out on? limitations to the problem (such as not creating the textures yet)

<h2>Extensions</h2>
<li>Virtual Textures and Virtual Geometry (Unreal Engine Nanite)</li>
<li>Synthetic Normal and Parallax Maps</li>
<li><a href="https://ieeexplore.ieee.org/document/8942651">Procedural Texture Generation (Tiny Glade)</a></li>
<li><a href="https://www.a23d.co/texture/syntheticmaterial-0400882">Synthetic Materials</a></li>


<h4><i>Sources::<i></h4>
<li><a href="https://courses.cs.washington.edu/courses/cse455/08wi/lectures/texture.pdf">University of Washington Texture Lecture</a></li>
<li><a href="https://graphics.cs.yale.edu/sites/default/files/edge-based_procedural_textures.pdf">Edge-Based Procedural Textures</a></li>
<li><a href="https://github.com/anopara">Anastasia Opara GitHub Research Repo</a></li>
<li><a href="https://medium.com/@allenaltiner/computer-generated-procedural-3d-and-multi-example-texture-based-synthesis-by-the-super-talented-4d1cfe34b05c">Texture Synthesis from Example</a></li>
<li><a href="https://pixel.ecn.purdue.edu:8443/projects/ITRweb/content/publication_related/uiuc/textureshop.pdf">University of Illinois - Textureshop</a></li>
<li><a href="https://apmonitor.com/pds/index.php/Main/TextureClassification">Machine Learning for Engineers - Prof. John D. Hedengren, BYU</a></li>